In [11]:
import numpy as np
from qiskit import QuantumCircuit, transpile,assemble
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Operator

In [12]:

def calculate_eigenvalues(sigma, t):
    b = np.exp(sigma * np.sqrt(1/t)) - 1
    return b

def build_quantum_op(b, r, t):
    a = -b  # Symmetric up and down movements
    A = np.array([[1+b, 0], [0, 1+a]])
    return A

# Parameters
sigma = 0.1
r = 0.05
T = 1
N = 3
t = T / N

b = calculate_eigenvalues(sigma, t)
A = build_quantum_op(b, r, t)
eigenvalues, eigenvectors = np.linalg.eigh(A)



In [22]:
qc = QuantumCircuit(N)

# Assuming the same dynamics on each qubit
for i in range(N):
    initial_state = eigenvectors[:, 0]  # Use the first eigenvector
    qc.initialize(initial_state, [i])  # Initialize each qubit independently


In [21]:
# now we need to prepare the intitial quantum state on the density matrix

# def init_quant_state(qc, eigenvectors, qubits):
#     # Initialize to the first eigenvector if it's dimensionally correct
#     if eigenvectors.shape[1] == 2**qubits:
#         initial_state = eigenvectors[:, 0]
#         qc.initialize(initial_state, range(qubits))
#     else:
#         print(eigenvectors.shape[1])
#         print(qubits)
#         #raise ValueError("The dimension of the eigenvector does not match the number of qubits.")

# qc=QuantumCircuit(N)
# init_quant_state(qc,eigenvectors,N)




2
3


In [23]:
def apply_stock_movements(qc, num_periods, up_theta, down_theta):
    for i in range(num_periods):
        qc.h(i)  # Create superposition
        qc.cry(up_theta, i, (i + 1) % num_periods)  # Controlled-RY for up movement
        qc.cry(down_theta, i, (i + 1) % num_periods)  # Controlled-RY for down movement

apply_stock_movements(qc, N, np.pi/4, -np.pi/4)
qc.measure_all()


In [25]:
simulator = AerSimulator()
transpiled_circuit = transpile(qc, simulator)
qobj = assemble(transpiled_circuit, shots=1000)
result = simulator.run(qobj).result()
counts = result.get_counts(qc)
print(counts)


{'100': 113, '110': 132, '011': 145, '000': 122, '001': 142, '010': 119, '101': 118, '111': 109}


C:\Users\chaha\AppData\Local\Temp\ipykernel_9516\2237938607.py:4: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.14 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = simulator.run(qobj).result()


In [26]:
#set strike price
K=50
def calculate_payoffs(final,K):
    payoffs = {k: max(int(k) - K, 0) for k in final_prices.keys()}
    return payoffs

# Assuming 'counts' is a dictionary where keys are final stock prices and values are their frequencies
final_prices = {'101': 500, '110': 300, '111': 200}  # Example output of quantum simulation
payoffs = calculate_payoffs(final_prices, K)
print(payoffs)

{'101': 51, '110': 60, '111': 61}
